In [1]:
# This is the code used to acquire the advanced team data
# Currently set up to run just one season at a time
# To run multiple seasons at once you can concatenate the game_id lists of multiple seasons


In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import boxscoreadvancedv2
import time

In [3]:
#Request information
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

# get game logs from the reg season, change season nullable to whatever season is desired
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2014-15',
                                               league_id_nullable='00',
                                               season_type_nullable='Regular Season')
games = gamefinder.get_data_frames()[0]

# get a list of distinct game ids
games_list = games['GAME_ID'].unique().tolist()


In [4]:
# Initailization for acquisition loop, only run first time through
# do not run if this if acquistion loop has run at least one game.
games_done = []
all_games = pd.DataFrame()

In [13]:
# Loop to acquire data
temp_games = []


for count ,i in enumerate(games_list):
    try:
        if i not in games_done:
            print(count,'game_id : ', i, ', Percent Done: ', round(((count+1)/len(games_list))*100,2) , '%')
            player_stats_data = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id= i, headers=headers, timeout=100)
            temp_stats_df = player_stats_data.get_data_frames()[1]
            temp_games.append(temp_stats_df)
            time.sleep(.6)
        else:
            pass
    except KeyboardInterrupt:
        break 
    except:
        print('skipped', i)
        pass
    


7 game_id :  0021401226 , Percent Done:  0.65 %
skipped 0021401226
15 game_id :  0021401216 , Percent Done:  1.3 %
16 game_id :  0021401215 , Percent Done:  1.38 %
17 game_id :  0021401210 , Percent Done:  1.46 %


In [10]:
# Run after above loop completes to concat our temp games dataframes into a master df
temp_games=pd.concat(temp_games)  

In [11]:
# Combine temporary datframes with our master all games df
# As well as create a list of games we have already run.
all_games = pd.concat([all_games,temp_games])
games_done = list(all_games['GAME_ID'])

In [12]:
# Verify that the length of the all_games game_id 
# is the same as the games list from the start
all_games = all_games.sort_values(['GAME_ID']).drop_duplicates(subset = ['TEAM_ID', 'GAME_ID'])
print(all_games.shape[0]/2, len(games_list))


14.0 1230


In [26]:
#Export to CSV
all_games.sort_values(['GAME_ID']).to_csv(r"Your Path Here", index=False)


In [ ]:
# Initializing our all_games variable
all_games = []

# This loop 
for count ,i in enumerate(games_list):
    print(count,'game_id : ', i, ', Percent Done: ', round(((count+1)/len(games_list))*100,2) , '%')
    player_stats_data = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id= i, headers=headers, timeout=100)
    stats_df = player_stats_data.get_data_frames()[1]
    all_games.append(stats_df)
    time.sleep(.600)
    
    
#run after above loop completes
all_games=pd.concat(all_games)  